In [ ]:
!pip install --upgrade langchain langchain-ollama langchain-community chromadb
!pip install fastapi nest-asyncio pyngrok uvicorn line-bot-sdk python-dotenv

In [2]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, Request, HTTPException
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

In [3]:
def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            documents.extend(loader.load())
    return documents

docs = load_documents("./")

In [4]:
llm = OllamaLLM(
    model="llama3.2",
    temperature=0.3,
)

embeddings = OllamaEmbeddings(
    model="llama3.2"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

split_docs = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever()

In [5]:
prompt_template = """
คุณเป็น ChatBot ให้คำปรึกษา ที่ชื่อว่า KidneyCare ผู้ช่วยให้คำปรึกษาที่มีความรู้ความเข้าใจลึกซึ้งในเรื่องการดูแลสุขภาพของผู้ป่วยโรคไต รวมถึงการแนะนำการดูแลตนเองหลังการผ่าตัดไต คุณสามารถตอบคำถามเกี่ยวกับโรคไต การดูแลสุขภาพ การรักษาโรคไต และการดูแลตนเองหลังการผ่าตัดไตได้ 
โดยคุณจะเชี่ยวชาญและแนะนำได้ดีเป็นพิเศษในเรื่องของอาหารการกินสำหรับผู้ป่วยโรคไต
ตอบคำถามต่อไปนี้โดยยึดตามข้อมูลที่ให้มาเท่านั้น และควรแน่ใจว่าคำตอบถูกต้องตามข้อมูลที่มีอยู่
ถ้าไม่มั่นใจว่ามีข้อมูลก็ควรบอกว่าขออภัยไม่สามารถตอบคำถามนี้ได้ คำถามของคุณจะถูกบันทึกไว้เพื่อปรับปรุงคุณภาพของ AI ของเรา
และตอบด้วยความชัดเจน กระชับ ในภาษาไทย ลงท้ายด้วยคำว่า "ค่ะ" เพื่อให้สุภาพ และแทนตัวเองด้วย "ฉัน" หากคำถามไหนเป็นคำถามง่าย ๆ สามารถตอบได้เลย ไม่ต้องให้คำตอบยาว ๆ แต่ถ้าผู้ใช้ต้องการรายละเอียด ให้ตอบรายละเอียดเพิ่มเติม แต่หากผู้ใช้ไม่บอกว่าต้องการรู้เยอะแค่ไหน ให้ถามก่อนเสมอ

บริบท: {context}

คำถาม: {question}

คำตอบ:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
def ask_question(question):
    result = qa_chain({"query": question})
    return result["result"]

question = "คุณคือใคร"
answer = ask_question(question)
print(f"คำถาม: {question}")
print(f"คำตอบ: {answer}")

In [ ]:
from dotenv import load_dotenv
from pyngrok import ngrok
import os

load_dotenv()
app = FastAPI()

line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

In [ ]:
import requests

def start_loading_animation(user_id):
    url = "https://api.line.me/v2/bot/chat/loading/start"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))}"
    }
    data = {"chatId": user_id}
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code != 200:
        print("Failed to start loading animation:", response.text)

@app.post("/callback")
async def callback(request: Request):
    signature = request.headers['X-Line-Signature']
    body = await request.body()

    try:
        handler.handle(body.decode("utf-8"), signature)
    except InvalidSignatureError:
        raise HTTPException(status_code=400, detail="Invalid signature")

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id  
    start_loading_animation(user_id)  
    
    text = event.message.text
    reply_text = ask_question(text)

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_text)
    )


ngrok_tunnel = ngrok.connect(8000)
print('Webhook URL:', ngrok_tunnel.public_url + '/callback')

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [70]:
!pgrep -af ngrok

71656


In [71]:
!killall ngrok